Import Package dan Dataset

In [ ]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [ ]:
test=pd.read_excel('drive/My Drive/Data BDC - Satria Data 2020/Data Uji/Data Uji BDC.xlsx')
train=pd.read_excel('drive/My Drive/Data BDC - Satria Data 2020/Data Latih/Data Latih BDC.xlsx')
testoriginal=test.copy()
trainoriginal=train.copy()
dataset=train.copy()
dataset=dataset.append(test)
dataset=dataset.reset_index()
dataset

,index,ID,label,tanggal,judul,narasi,nama file gambar
0,0,71,1.0,2020-08-17 00:00:00,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,1,461,1.0,2020-07-17 00:00:00,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,2,495,1.0,2020-07-13 00:00:00,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,3,550,1.0,2020-07-08 00:00:00,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,4,681,1.0,2020-06-24 00:00:00,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg
...,...,...,...,...,...,...,...
4696,465,206021,NaN,2020-05-01 00:00:00,"Dijemput oleh Tim Medis, Kakak Beradik Usia 8 ...","DiTangerang, dua bersaudara kakak beradik usia...",206021.jpg
4697,466,219874,NaN,2017-10-02 00:00:00,": Pertanda Alam, Kera Putih Mulai Turun Dari G...","Pertanda Alam, Kera Putih Mulai Turun Dari Gun...",219874.jpg
4698,467,222802,NaN,2020-03-22 00:00:00,Pasar Wameo dan Karya Nugraha di Tutup selama ...,Assalamu Alaikum …\n \n Info dari pedagang pa...,222802.jpg
4699,468,226020,NaN,2018-09-25 00:00:00,Klarifikasi KPK Terkait Foto Setya Novanto Tan...,Secara tidak sengaja sekitar pukul 06.00 WIB d...,226020.jpg


Penghilangan simbol dan perubahan menjadi huruf kecil pada kolom narasi dan judul

In [ ]:
#ilangin simbol
dataset['narasibersih'] = dataset['narasi'].str.replace("[^a-zA-Z#]", " ")
dataset['narasibersih'] = dataset['narasibersih'].str.lower()
dataset['judulbersih'] = dataset['judul'].str.replace("[^a-zA-Z#]", " ")
dataset['judulbersih'] = dataset['judulbersih'].str.lower()
dataset.head(10)

,index,ID,label,tanggal,judul,narasi,nama file gambar,narasibersih,judulbersih
0,0,71,1.0,2020-08-17 00:00:00,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg,a caller to a radio talk show recently shared ...,pemakaian masker menyebabkan penyakit legionna...
1,1,461,1.0,2020-07-17 00:00:00,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png,yth seluruh anggota grup sesuai instruksi gube...,instruksi gubernur jateng tentang penilangan ...
2,2,495,1.0,2020-07-13 00:00:00,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png,jokowi adalah presiden terbaik dlm sejarah ban...,foto jim rohn jokowi adalah presiden terbaik ...
3,3,550,1.0,2020-07-08 00:00:00,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png,maaf mas dan mbak ini bukan politik tapi k...,ini bukan politik tapi kenyataan pak jokowi b...
4,4,681,1.0,2020-06-24 00:00:00,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg,kadrun kalo lihat foto ini panas dingin,foto kadrun kalo lihat foto ini panas dingin
5,5,736,1.0,2020-06-17 00:00:00,event promo smartphone JNE 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma...",736.png,selamat siang teman teman fb ku semuanya cuma...,event promo smartphone jne spesial di bul...
6,6,1161,1.0,2020-05-05 00:00:00,MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UN...,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGA...,1161.png,lihatlah mereka sudah mempersiapkan diri denga...,mereka sudah mempersiapkan diri dengan baik un...
7,7,1597,1.0,2020-03-24 00:00:00,Foto Presiden Italia menangis karena tak cukup...,Italia punya fasilitas perawatan kesehatan ter...,1597.png,italia punya fasilitas perawatan kesehatan ter...,foto presiden italia menangis karena tak cukup...
8,8,2098,1.0,2020-01-09 00:00:00,"Kapolres Timor Tengah Utara , Nusa Tenggara Ti...",Polisi goblok. Bukanya cpet” d tolongin malah ...,2098.jpg,polisi goblok bukanya cpet d tolongin malah ...,kapolres timor tengah utara nusa tenggara ti...
9,9,2226,1.0,2019-12-24 00:00:00,Video Polisi china telah menganiaya wanita uig...,Polisi china telah menganiaya wanita uighur le...,2226.png,polisi china telah menganiaya wanita uighur le...,video polisi china telah menganiaya wanita uig...


Import Package untuk Penerjemahan data

In [ ]:
pip install googletrans

     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 972kB 9.6MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 102kB 7.6MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=84cf81581773637abd521eab2c4182dc8c33fff27154b9cc9f5eaa416c79acca
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=a00ea0b46fdf865bc88caefc80469ced5e95121ed002401f87f30ac28d52db44
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [ ]:
from googletrans import Translator
import time
translator = Translator()
narasi=dataset['narasibersih']
judul=dataset['judulbersih']

In [ ]:
for i in range (len(narasi)):
  translation = translator.translate(narasi[i], dest='id')
  if translation.src=='en':
    dataset.iloc[[i],[5]]=translation.text
    print(i, translation.origin, '->',translation.text)
  if i%400==0:
    time.sleep(10)
    translator = Translator()

0 a caller to a radio talk show recently shared that his wife was hospitalized n told she had covid n only a couple of days left to live   a doctor friend suggested she be tested for legionnaires disease because she wore the same mask every day all day long   turns out it was legionnaires disease from the moisture n bacteria in her mask   she was given antibiotics n within two days was better   what if these  spikes  in covid are really something else due to  mask induced infections         -> seorang penelepon ke acara bincang-bincang radio baru-baru ini berbagi bahwa istrinya dirawat di rumah sakit dan diberitahu bahwa dia telah terinfeksi dan hanya tinggal beberapa hari lagi seorang teman dokter menyarankan dia untuk menjalani tes penyakit legiuner karena dia mengenakan masker yang sama setiap hari sepanjang hari. di luar itu penyakit legiuner dari kelembaban n bakteri di masker dia diberi antibiotik dan dalam dua hari lebih baik bagaimana jika lonjakan pada covid ini benar-benar se

In [ ]:
for i in range (len(judul)):
  translation = translator.translate(judul[i], dest='id')
  if translation.src=='en':
    dataset.iloc[[i],[6]]=translation.text
    print(i, translation.origin, '->',translation.text)
  if i%400==0:
    time.sleep(10)
    translator = Translator()

152 video in america there has been looting everywhere for fear of the corona virus -> Video di Amerika telah terjadi penjarahan dimana-mana karena takut akan virus corona
169 a zambian student in china was beaten to death -> seorang mahasiswa zambia di cina dipukuli sampai mati
188 miscaptioned wolfpack photo -> foto paket serigala salah kaprah
217  g antenna in your mask -> g antena di topeng Anda
245 the un  oic  world human rights organization have become blind   -> organisasi hak asasi manusia duniawi telah menjadi buta
331 some say the sound of a whale -> beberapa mengatakan suara ikan paus
390 wearing a mask causes legionnaires disease -> memakai topeng menyebabkan penyakit legiuner
434      italian magazine described      world would look like -> majalah italia menggambarkan dunia akan terlihat seperti
436 jack mas advice  hopefully we can get through      -> semoga saran mas jack bisa kita lalui
493 us government funded insect spy drone and already in production -> kami drone 

Menghapus kata yang terdiri dari 2 huruf atau kurang serta tokenisasi

In [ ]:
#lowercase
dataset['narasibersih'] = dataset['narasibersih'].str.lower()
dataset['judulbersih'] = dataset['judulbersih'].str.lower()
#ilangin kata yang panjangnya kurang dari 2 huruf
dataset['narasibersih'] = dataset['narasibersih'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
dataset['judulbersih'] = dataset['judulbersih'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
dataset.head()

,index,ID,label,tanggal,judul,narasi,nama file gambar,narasibersih,judulbersih
0,0,71,1.0,2020-08-17 00:00:00,Pemakaian Masker Menyebabkan Penyakit Legionna...,seorang penelepon ke acara bincang-bincang rad...,71.jpg,caller radio talk show recently shared that hi...,pemakaian masker menyebabkan penyakit legionna...
1,1,461,1.0,2020-07-17 00:00:00,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png,yth seluruh anggota grup sesuai instruksi gube...,instruksi gubernur jateng tentang penilangan b...
2,2,495,1.0,2020-07-13 00:00:00,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png,jokowi adalah presiden terbaik dlm sejarah ban...,foto jim rohn jokowi adalah presiden terbaik d...
3,3,550,1.0,2020-07-08 00:00:00,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png,maaf mas dan mbak ini bukan politik tapi kenya...,ini bukan politik tapi kenyataan pak jokowi be...
4,4,681,1.0,2020-06-24 00:00:00,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg,kadrun kalo lihat foto ini panas dingin,foto kadrun kalo lihat foto ini panas dingin


In [ ]:
#tokenize 
tokenized_narasi = dataset['narasibersih'].apply(lambda x: x.split())
tokenized_judul = dataset['judulbersih'].apply(lambda x: x.split())
tokenized_judul.head()

0    [pemakaian, masker, menyebabkan, penyakit, leg...
1    [instruksi, gubernur, jateng, tentang, penilan...
2    [foto, jim, rohn, jokowi, adalah, presiden, te...
3    [ini, bukan, politik, tapi, kenyataan, pak, jo...
4    [foto, kadrun, kalo, lihat, foto, ini, panas, ...
Name: judulbersih, dtype: object

In [ ]:
tokenized_narasi.head()

0    [caller, radio, talk, show, recently, shared, ...
1    [yth, seluruh, anggota, grup, sesuai, instruks...
2    [jokowi, adalah, presiden, terbaik, dlm, sejar...
3    [maaf, mas, dan, mbak, ini, bukan, politik, ta...
4      [kadrun, kalo, lihat, foto, ini, panas, dingin]
Name: narasibersih, dtype: object

In [ ]:
print(len(tokenized_narasi))
print(len(tokenized_judul))

4701
4701


Autocorrect

In [ ]:
import io
import time
from datetime import timedelta
import logging
logging.basicConfig(level=logging.INFO)
import gensim
from gensim.models.wrappers import FastText
import shutil

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [ ]:
#downloading model
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz

In [ ]:
import os
os.mkdir('model')

In [ ]:
shutil.copy2('drive/My Drive/cc.id.300.bin.gz','model')

'model/cc.id.300.bin.gz'

In [ ]:
!gunzip 'model/cc.id.300.bin.gz'

In [ ]:
model=FastText.load_fasttext_format('model/cc.id.300.bin')

INFO:gensim.models.deprecated.fasttext_wrapper:loading 2000000 words for fastText model from model/cc.id.300.bin
INFO:gensim.models.deprecated.fasttext_wrapper:loading weights for 2000000 words for fastText model from model/cc.id.300.bin
INFO:gensim.models.deprecated.fasttext_wrapper:loaded (2000000, 300) weight matrix for fastText model from model/cc.id.300.bin


In [ ]:
words=list(model.wv.vocab)
w_rank={}
for i,word in enumerate (words):
  w_rank[word]=i
WORDS=w_rank

In [ ]:
import itertools
dict(itertools.islice(WORDS.items(),10))

{'"': 5,
 '(': 8,
 ')': 7,
 ',': 0,
 '.': 1,
 '</s>': 2,
 'dan': 4,
 'dengan': 9,
 'di': 6,
 'yang': 3}

In [ ]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+',text.lower())

def P(word):
  return - WORDS.get(word,0)

def correction(word):
  return max(candidates(word),key=P)

def candidates(word):
  return (known([word]) or known(edits1(word)) or known (edits2(word)) or [word])

def known(words):
  return set(w for w in words if w in WORDS)

def edits1(word):
  letters='abcdefghijklmnopqrstuvwxyz'
  splits=[(word[:i], word[i:]) for i in range(len(word)+1)]
  deletes=[L+R[1:] for L, R in splits if R]
  transposes=[L+R[1]+R[0]+R[2:] for L, R in splits if len(R)>1]
  replaces=[L+c+R[1:] for L, R in splits if R for c in letters]
  inserts=[L+c+R for L, R in splits for c in letters]
  return set(deletes+transposes+replaces+inserts)

def edits2(word):
  return(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
#Autocorrection
for i in range(len(tokenized_narasi)):
  katadasar=[]
  kalimat = tokenized_narasi[i]
  for j in kalimat:
    katadasar.append(correction(j))
  print('baris ke-',i)
  tokenized_narasi[i] = katadasar
tokenized_narasi.head()

baris ke- 0
baris ke- 1
baris ke- 2
baris ke- 3
baris ke- 4
baris ke- 5
baris ke- 6
baris ke- 7
baris ke- 8
baris ke- 9
baris ke- 10
baris ke- 11
baris ke- 12
baris ke- 13
baris ke- 14
baris ke- 15
baris ke- 16
baris ke- 17
baris ke- 18
baris ke- 19
baris ke- 20
baris ke- 21
baris ke- 22
baris ke- 23
baris ke- 24
baris ke- 25
baris ke- 26
baris ke- 27
baris ke- 28
baris ke- 29
baris ke- 30
baris ke- 31
baris ke- 32
baris ke- 33
baris ke- 34
baris ke- 35
baris ke- 36
baris ke- 37
baris ke- 38
baris ke- 39
baris ke- 40
baris ke- 41
baris ke- 42
baris ke- 43
baris ke- 44
baris ke- 45
baris ke- 46
baris ke- 47
baris ke- 48
baris ke- 49
baris ke- 50
baris ke- 51
baris ke- 52
baris ke- 53
baris ke- 54
baris ke- 55
baris ke- 56
baris ke- 57
baris ke- 58
baris ke- 59
baris ke- 60
baris ke- 61
baris ke- 62
baris ke- 63
baris ke- 64
baris ke- 65
baris ke- 66
baris ke- 67
baris ke- 68
baris ke- 69
baris ke- 70
baris ke- 71
baris ke- 72
baris ke- 73
baris ke- 74
baris ke- 75
baris ke- 76
baris ke-

0    [caller, radio, talk, show, recently, shared, ...
1    [yth, seluruh, anggota, grup, sesuai, instruks...
2    [jokowi, adalah, presiden, terbaik, dlm, sejar...
3    [maaf, mas, dan, mbak, ini, bukan, politik, ta...
4       [karun, kalo, lihat, foto, ini, panas, dingin]
Name: narasibersih, dtype: object

In [ ]:
#Autocorrection
for i in range(len(tokenized_judul)):
  katadasar=[]
  kalimat = tokenized_judul[i]
  for j in kalimat:
    katadasar.append(correction(j))
  print('baris ke-',i)
  tokenized_judul[i] = katadasar
tokenized_judul.head()

baris ke- 0
baris ke- 1
baris ke- 2
baris ke- 3
baris ke- 4
baris ke- 5
baris ke- 6
baris ke- 7
baris ke- 8
baris ke- 9
baris ke- 10
baris ke- 11
baris ke- 12
baris ke- 13
baris ke- 14
baris ke- 15
baris ke- 16
baris ke- 17
baris ke- 18
baris ke- 19
baris ke- 20
baris ke- 21
baris ke- 22
baris ke- 23
baris ke- 24
baris ke- 25
baris ke- 26
baris ke- 27
baris ke- 28
baris ke- 29
baris ke- 30
baris ke- 31
baris ke- 32
baris ke- 33
baris ke- 34
baris ke- 35
baris ke- 36
baris ke- 37
baris ke- 38
baris ke- 39
baris ke- 40
baris ke- 41
baris ke- 42
baris ke- 43
baris ke- 44
baris ke- 45
baris ke- 46
baris ke- 47
baris ke- 48
baris ke- 49
baris ke- 50
baris ke- 51
baris ke- 52
baris ke- 53
baris ke- 54
baris ke- 55
baris ke- 56
baris ke- 57
baris ke- 58
baris ke- 59
baris ke- 60
baris ke- 61
baris ke- 62
baris ke- 63
baris ke- 64
baris ke- 65
baris ke- 66
baris ke- 67
baris ke- 68
baris ke- 69
baris ke- 70
baris ke- 71
baris ke- 72
baris ke- 73
baris ke- 74
baris ke- 75
baris ke- 76
baris ke-

0    [pemakaian, masker, menyebabkan, penyakit, leg...
1    [instruksi, gubernur, jateng, tentang, penilan...
2    [foto, jim, roh, jokowi, adalah, presiden, ter...
3    [ini, bukan, politik, tapi, kenyataan, pak, jo...
4    [foto, karun, kalo, lihat, foto, ini, panas, d...
Name: judulbersih, dtype: object

In [ ]:
tokenized_narasi.head()

0    [caller, radio, talk, show, recently, shared, ...
1    [yth, seluruh, anggota, grup, sesuai, instruks...
2    [jokowi, adalah, presiden, terbaik, dlm, sejar...
3    [maaf, mas, dan, mbak, ini, bukan, politik, ta...
4       [karun, kalo, lihat, foto, ini, panas, dingin]
Name: narasibersih, dtype: object

In [ ]:
tokenized_judul.head()

0    [pemakaian, masker, menyebabkan, penyakit, leg...
1    [instruksi, gubernur, jateng, tentang, penilan...
2    [foto, jim, roh, jokowi, adalah, presiden, ter...
3    [ini, bukan, politik, tapi, kenyataan, pak, jo...
4    [foto, karun, kalo, lihat, foto, ini, panas, d...
Name: judulbersih, dtype: object

In [ ]:
for i in range(len(tokenized_narasi)):
    tokenized_narasi[i] = ' '.join(tokenized_narasi[i])
for i in range(len(tokenized_judul)):
    tokenized_judul[i] = ' '.join(tokenized_judul[i])

dataset['narasibersih'] = tokenized_narasi
dataset['judulbersih'] = tokenized_judul
dataset.head()

,index,ID,label,tanggal,judul,narasi,nama file gambar,narasibersih,judulbersih
0,0,71,1.0,2020-08-17 00:00:00,Pemakaian Masker Menyebabkan Penyakit Legionna...,seorang penelepon ke acara bincang-bincang rad...,71.jpg,caller radio talk show recently shared that hi...,pemakaian masker menyebabkan penyakit legionna...
1,1,461,1.0,2020-07-17 00:00:00,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png,yth seluruh anggota grup sesuai instruksi gube...,instruksi gubernur jateng tentang penilangan b...
2,2,495,1.0,2020-07-13 00:00:00,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png,jokowi adalah presiden terbaik dlm sejarah ban...,foto jim roh jokowi adalah presiden terbaik dl...
3,3,550,1.0,2020-07-08 00:00:00,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png,maaf mas dan mbak ini bukan politik tapi kenya...,ini bukan politik tapi kenyataan pak jokowi be...
4,4,681,1.0,2020-06-24 00:00:00,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg,karun kalo lihat foto ini panas dingin,foto karun kalo lihat foto ini panas dingin


Fitur Ekstraksi Bag of Words

In [ ]:
# Download Stopwords indonesia
nltk.download('stopwords')
swords=nltk.corpus.stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=3, max_features=1000, stop_words = swords)
bow1 = bow_vectorizer.fit_transform(dataset['narasibersih'])
print(bow_vectorizer.get_feature_names())
bow2 = bow_vectorizer.fit_transform(dataset['judulbersih'])
print(bow_vectorizer.get_feature_names())

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
df_bulan=pd.DataFrame(arrbulantest)
df_tanggal=pd.DataFrame(arrtanggaltest)
df_tahun=pd.DataFrame(arrtahuntest)
df_bulan.shape

(4701, 1)

In [ ]:
df_bow1 = pd.DataFrame(bow1.todense())
df_bow2 = pd.DataFrame(bow2.todense())
df_bow=pd.concat([df_tanggal,df_bulan,df_tahun,df_bow1,df_bow2],axis=1)
df_bow

,0,0,0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,17,8,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,17,7,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13,7,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8,7,2020,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,24,6,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4696,1,5,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4697,2,10,2017,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4698,22,3,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4699,25,9,2018,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
df_bow=sc_X.fit_transform(df_bow)

In [ ]:
dfbowtest=df_bow[4231:,:]
dfbowtrain=df_bow[:4231,:]
print(len(dfbowtest))
print(len(dfbowtrain))
trainlabel=pd.DataFrame(train['label'])